# Article Text Summarization

## NLTK Approach

In [1]:
import nltk
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# The low end of shared words to consider
LOWER_BOUND = .20

# The high end, since anything above this is probably SEO garbage or a
# duplicate sentence
UPPER_BOUND = .90

def is_unimportant(word):
    """Decides if a word is ok to toss out for the sentence comparisons"""
    return word in ['.', '!', ',', ] or '\'' in word or word in stop_words

def only_important(sent):
    """Just a little wrapper to filter on is_unimportant"""
    return filter(lambda w: not is_unimportant(w), sent)

def compare_sents(sent1, sent2):
    """Compare two word-tokenized sentences for shared words"""
    if not len(sent1) or not len(sent2):
        return 0
    return len(set(only_important(sent1)) & set(only_important(sent2))) / ((len(sent1) + len(sent2)) / 2.0)

def compare_sents_bounded(sent1, sent2):
    """If the result of compare_sents is not between LOWER_BOUND and
    UPPER_BOUND, it returns 0 instead, so outliers don't mess with the sum"""
    cmpd = compare_sents(sent1, sent2)
    if LOWER_BOUND < cmpd < UPPER_BOUND:
        return cmpd
    else:
        return 0

def compute_score(sent, sents):
    """Computes the average score of sent vs the other sentences (the result of
    sent vs itself isn't counted because it's 1, and that's above
    UPPER_BOUND)"""
    if not len(sent):
        return 0
    return sum(compare_sents_bounded(sent, sent1) for sent1 in sents) / float(len(sents))

def summarize(block):
    """Return the sentence that best summarizes block"""
    if not block:
        return None
    sents = nltk.sent_tokenize(block)
    word_sents = list(map(nltk.word_tokenize, sents))
    d = dict((compute_score(word_sent, word_sents), sent)
             for sent, word_sent in zip(sents, word_sents))
    return d[max(d.keys())]

In [2]:
article = """
Taking vitamin C orally along with conventional antihypertensive medications appears to modestly decrease systolic blood pressure, but the effects on diastolic pressure are mixed . In patients with type 2 diabetes, taking vitamin C 500 mg daily for 4 weeks, in addition to antihypertensives, seems to reduce arterial blood pressure and decrease arterial stiffness . However, taking supplemental vitamin C 500 mg daily without antihypertensives does not seem to reduce systolic or diastolic blood pressure . Dietary restriction of vitamin C is associated with increases in both diastolic and systolic blood pressure (10354
"""
summarize(article)

'However, taking supplemental vitamin C 500 mg daily without antihypertensives does not seem to reduce systolic or diastolic blood pressure .'

## ML Approach

### Getting the Text ready

In [3]:
from neo4j.v1 import GraphDatabase, basic_auth
from collections import Counter

driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "naturalmed"))
session = driver.session()

In [42]:
query = session.run("MATCH (m:Medicine)-[r1]->(n:DiseaseInteraction)-[r2]->(d:Disease) \
                     RETURN m.name, n.text, d.id")

data = [(r['m.name'], r['n.text'], r['d.id'], summarize(r['n.text'])) for r in query]

def tkz(s):
    return " ".join(nltk.tokenize.word_tokenize(s))

summaries, texts, dumb = zip(*[(tkz(e[3]), tkz(e[1]), None) for e in data])

In [46]:
texts[1]

'Constituents isolated from Acacia rigidula are thought to have stimulant effects . Additionally , dietary supplements listing Acacia rigidula as an ingredient have been shown to contain the stimulant beta-methylphenethylamine ( BMPEA ) and greater amounts of other stimulant compounds than would normally be found in plant parts . BMPEA has been shown to increase blood pressure and heart rate in animals . Theoretically , taking Acacia rigidula supplements might exacerbate high blood pressure .'

In [45]:
summaries[1]

'Theoretically , taking Acacia rigidula supplements might exacerbate high blood pressure .'

In [48]:
len(texts)

2336

In [47]:
len(texts) == len(summaries) == len(dumb)

True

In [49]:
tokenized_data = (summaries, texts, dumb)

### Dumping the objects to the disk

In [50]:
import pickle

FN = 'natmed_tokens'

with open('./../Dumps/%s.data.pkl'%FN,'wb') as fp:
    pickle.dump(tokenized_data,fp)

### Starting Building the Model

In [44]:
#session.close()